# Library

In [2]:
import pandas as pd
import os
import numpy as np
from datetime import datetime
import pyexasol

In [3]:
pd.options.display.float_format = '{:.2f}'.format
# pd.reset_option('display.float_format')

In [4]:
# 경고문 없애는 코드
# UserWarning: Workbook contains no default style, apply openpyxl's default warn("Workbook contains no default style, apply openpyxl's default")
import warnings
warnings.simplefilter("ignore")

# Fold

In [5]:
pub_fold = 'D:/data/processing/big3/raw/public'
raw_fold = 'D:/data/processing/big3/raw'
raw2_fold = 'D:/data/processing/big3/raw/BD2'
raw4_fold = 'D:/data/processing/big3/raw/BD4'
raw5_fold = 'D:/data/processing/big3/raw/BD5'
an_fold = 'D:/data/processing/big3/analysis'
an2_fold = 'D:/data/processing/big3/analysis/BD2'
an4_fold = 'D:/data/processing/big3/analysis/BD4'
an5_fold = 'D:/data/processing/big3/analysis/BD5'
df_fold = 'D:/data/processing/big3/df'
df2_fold = 'D:/data/processing/big3/df/BD2'
df4_fold = 'D:/data/processing/big3/df/BD4'
df5_fold = 'D:/data/processing/big3/df/BD5'
folds = {
    'pub_fold':pub_fold, 
    'raw_fold':raw_fold, 
    'raw2_fold':raw2_fold, 
    'raw4_fold':raw4_fold, 
    'raw5_fold':raw5_fold, 
    'an_fold':an_fold, 
    'an2_fold:':an2_fold, 
    'an4_fold:':an4_fold, 
    'an5_fold:':an5_fold, 
    'df_fold':df_fold, 
    'df2_fold':df2_fold, 
    'df4_fold':df4_fold, 
    'df5_fold':df5_fold, 
    }
folds

{'pub_fold': 'D:/data/processing/big3/raw/public',
 'raw_fold': 'D:/data/processing/big3/raw',
 'raw2_fold': 'D:/data/processing/big3/raw/BD2',
 'raw4_fold': 'D:/data/processing/big3/raw/BD4',
 'raw5_fold': 'D:/data/processing/big3/raw/BD5',
 'an_fold': 'D:/data/processing/big3/analysis',
 'an2_fold:': 'D:/data/processing/big3/analysis/BD2',
 'an4_fold:': 'D:/data/processing/big3/analysis/BD4',
 'an5_fold:': 'D:/data/processing/big3/analysis/BD5',
 'df_fold': 'D:/data/processing/big3/df',
 'df2_fold': 'D:/data/processing/big3/df/BD2',
 'df4_fold': 'D:/data/processing/big3/df/BD4',
 'df5_fold': 'D:/data/processing/big3/df/BD5'}

# server

In [6]:
# exasol db
wd = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSD')
we = pyexasol.connect(dsn='dev.openankus.org:8563', user='sys', password='djslzja', compression=True, schema='VSYSE')

# 로드

## 로드-등록정보

In [70]:
# 5m 31.4s
car = wd.export_to_pandas("SELECT VIN, BSPL_STDG_CD, EXHST_GAS_GRD_CD, EXHST_GAS_CERT_NO, VHCL_ERSR_YN, MANP_MNG_NO, YRIDNW, VHCTY_CD, PURPS_CD2, FRST_REG_YMD, VHCL_FBCTN_YMD, VHCL_MNG_NO, VHRNO, EXTGAS_INSP_VLD_YMD, VHCL_OWNR_CL_CD FROM STD_CEG_CAR_MIG;")
car_ch_col = {
    'VIN':'차대번호', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_ERSR_YN':'차량말소여부',
    'MANP_MNG_NO':'제원관리번호', 
    'YRIDNW':'차량연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'FRST_REG_YMD':'최초등록일자',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'자동차등록번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',  
}
carr = car.rename(columns=car_ch_col)

print(carr.shape)
print('data load : STD_CEG_CAR_MIG')

(31131025, 15)
data load : STD_CEG_CAR_MIG


In [71]:
print(carr['배출가스등급'].unique())
## 배출가스등급 코드 변환
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
carr['배출가스등급'] = carr['배출가스등급'].replace(grd_dict)
print(carr['배출가스등급'].unique())

print(carr['차종'].unique())
# 4.0s
## 차종 코드 변환
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
carr['차종'] = carr['차종'].replace(cd_dict)
print(carr['차종'].unique())

print(carr['용도'].unique())
## 용도 코드 변환
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
carr['용도'] = carr['용도'].replace(purps_dict)
print(carr['용도'].unique())

['A0503' 'A0502' 'A0505' 'A0501' 'A0504' nan]
['3' '2' '5' '1' '4' nan]
['A31P' 'A31T' 'A31V' 'A31S']
['승용' '화물' '승합' '특수']
['A08P' 'A08B' 'A08O']
['개인용' '영업용' '관용']


## 로드-제원정보

In [8]:
# 4.2s
src = wd.export_to_pandas("SELECT MANP_MNG_NO, FUEL_CD, VHCTY_TY_CD2, MNFCTR_NM, VHCNM, VHCL_FRM, EGIN_TY, VHCTY_CL_CD, TOTL_WGHT, CRYNG_WGHT, DSPLVL, EGIN_OTPT FROM STD_CEG_CAR_SRC_MIG;")
src_ch_col = {
    'MANP_MNG_NO':'제원관리번호', 
    'FUEL_CD':'연료',
    'VHCTY_TY_CD2':'차종유형', 
    'MNFCTR_NM':'제작사명', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'VHCTY_CL_CD':'차종분류',
    'TOTL_WGHT':'총중량',
    'CRYNG_WGHT':'적재중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',
}
srcr = src.rename(columns=src_ch_col)

print(srcr.shape)
print('data load : STD_CEG_CAR_SRC_MIG')

(574467, 12)
data load : STD_CEG_CAR_SRC_MIG


In [ ]:
print(srcr['연료'].unique())
# 7.9s
## 연료 코드 변환
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
srcr['연료'] = srcr['연료'].replace(fuel_dict)
print(srcr['연료'].unique())

print(srcr['차종유형'].unique())
## 차종유형 코드 변환
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
srcr['차종유형'] = srcr['차종유형'].replace(ty_dict)
print(srcr['차종유형'].unique())

## 법정동코드

In [19]:
# 1.3s
code = wd.export_to_pandas("SELECT STDG_CD, STDG_CTPV_NM, STDG_SGG_NM, STDG_CTPV_CD, STDG_SGG_CD FROM STD_BJCD_INFO;")
code_ch_col = {
    'STDG_CD':'법정동코드', 
    'STDG_CTPV_NM':'시도', 
    'STDG_SGG_NM':'시군구', 
    'STDG_CTPV_CD':'시도코드',
    'STDG_SGG_CD':'시군구코드', 
}
coder = code.rename(columns=code_ch_col)

print(coder.columns)
print(coder.shape)
print('data load : STD_BJCD_INFO')

Index(['법정동코드', '시도', '시군구', '시도코드', '시군구코드'], dtype='object')
(47916, 5)
data load : STD_BJCD_INFO


## 로드-LST

In [100]:
# 3m 15.1s
cs_raw_dict = {
    'CLS_YMD':'원천데이터생성일자',
    
    'VIN':'차대번호', 
    'MANP_MNG_NO':'제원관리번호', 
    'VHCL_ERSR_YN':'차량말소여부',
    'FRST_REG_YMD':'최초등록일자',
    'YRIDNW':'연식', 
    'VHCTY_CD':'차종', 
    'PURPS_CD2':'용도', 
    'BSPL_STDG_CD':'법정동코드', 
    'EXHST_GAS_GRD_CD':'배출가스등급', 
    'FUEL_CD':'연료',
    'MNFCTR_NM':'제작사명', 
    'VHCTY_TY_CD2':'차종유형', 
    'VHCNM':'차명', 
    'VHCL_FRM':'자동차형식', 
    'EGIN_TY':'엔진형식', 
    'TOTL_WGHT':'총중량',
    'DSPLVL':'배기량', 
    'EGIN_OTPT':'엔진출력',

    'EXHST_GAS_CERT_NO':'배출가스인증번호',
    'VHCL_FBCTN_YMD':'제작일자', 
    'VHCL_MNG_NO':'차량관리번호', 
    'VHRNO':'자동차등록번호',
    'EXTGAS_INSP_VLD_YMD':'검사유효일',
    'VHCL_OWNR_CL_CD':'소유자구분',

    'VHCTY_CL_CD':'차종분류',
    'CRYNG_WGHT':'적재중량',

    'STDG_CTPV_NM':'시도명',
    'STDG_SGG_NM':'시군구명',
    'LEM_KND_CD':'저공해조치종류코드',
    'STRCT_CHG_YN':'구조변경여부',
    'FUEL_G_NM':'연료_G_명',
    'TKCAR_PSCP':'승차정원',
    'OVRLH':'전장',
    'OVRWD':'전폭',
    'OVRHG':'전고',
    'FBCTN_NTN_NM':'제작국명',
    'BOHUM_YN':'보훈여부',
}

name = 'STD_CEG_GRD_LST_202312_20240104'
file_name = f'{name}.csv'
cs_raw = pd.read_csv(os.path.join(pub_fold, file_name), low_memory=False, usecols=cs_raw_dict.keys())
cs = cs_raw.rename(columns=cs_raw_dict)
print(cs.columns)
print(cs.shape)
print('data load : STD_CEG_GRD_LST')

Index(['원천데이터생성일자', '차량관리번호', '자동차등록번호', '법정동코드', '시도명', '시군구명', '소유자구분', '차종',
       '용도', '차대번호', '최초등록일자', '연식', '제작일자', '검사유효일', '배출가스인증번호', '배출가스등급',
       '저공해조치종류코드', '구조변경여부', '차량말소여부', '제원관리번호', '차명', '차종분류', '차종유형',
       '자동차형식', '제작사명', '연료', '연료_G_명', '엔진형식', '총중량', '적재중량', '엔진출력', '배기량',
       '승차정원', '전장', '전폭', '전고', '제작국명', '보훈여부'],
      dtype='object')
(29709609, 38)
data load : STD_CEG_GRD_LST


In [101]:
# 코드 변환
## 배출가스등급 코드 변환
print(cs['배출가스등급'].unique())
grd_dict = {
    'A0501':'1', 
    'A0502':'2', 
    'A0503':'3', 
    'A0504':'4', 
    'A0505':'5', 
    'A05T2':'2',
    'A05T3':'3', 
    'A05T4':'4', 
    'A05T5':'5', 
    'A05X':'X', 
}
cs['배출가스등급'] = cs['배출가스등급'].replace(grd_dict)
print(cs['배출가스등급'].unique())

## 차종 코드 변환
print(cs['차종'].unique())
cd_dict = {
    'A31M':'이륜', 
    'A31P':'승용', 
    'A31S':'특수', 
    'A31T':'화물', 
    'A31V':'승합'
}
cs['차종'] = cs['차종'].replace(cd_dict)
print(cs['차종'].unique())

## 용도 코드 변환
print(cs['용도'].unique())
purps_dict = {
    'A08P':'개인용', 
    'A08B':'영업용', 
    'A08O':'관용',
}
cs['용도'] = cs['용도'].replace(purps_dict)
cs['용도'].unique()

## 소유자구분 코드 변환
print(cs['소유자구분'].unique())
ownr_dict = {
    'A27B':'사업자', 
    'A27F':'외국인', 
    'A27L':'법인', 
    'A27O':'기타', 
    'A27R':'주민', 
}
cs['소유자구분'] = cs['소유자구분'].replace(ownr_dict)
print(cs['소유자구분'].unique())

## 연료 코드 변환
print(cs['연료'].unique())
fuel_dict = {
    'A90GS':'휘발유', 
    'A91DS':'경유',
    'A92LP':'LPG(액화석유가스)', 
    'A90GH':'휘발유 하이브리드', 
    'A93EV':'전기', 
    'A91DH':'경유 하이브리드', 
    'A92CN':'CNG(압축천연가스)', 
    'A93HD':'수소', 
    'A92LH':'LPG 하이브리드', 
    'A94OT':'기타연료', 
    'A92CH':'CNG 하이브리드',
    'A90AC':'알코올', 
    'A93SH':'태양열', 
    'A91KS':'등유', 
    'A92LN':'LNG(액화천연가스)', 
    'A90PH':'플러그인 하이브리드', 
}
cs['연료'] = cs['연료'].replace(fuel_dict)
print(cs['연료'].unique())

## 차종유형 코드 변환
print(cs['차종유형'].unique())
ty_dict = {
    'A30C':'경형', 
    'A30L':'대형', 
    'A30M':'중형', 
    'A30S':'소형',
}
cs['차종유형'] = cs['차종유형'].replace(ty_dict)
print(cs['차종유형'].unique())

['A0505' 'A0503' 'A0504' 'A0502' 'X' 'A30M' 'A0501']
['5' '3' '4' '2' 'X' 'A30M' '1']
['A31T' 'A31P' 'A31V' 'A31S' 'YMY-HD-51']
['화물' '승용' '승합' '특수' 'YMY-HD-51']
['A08P' 'A08B' 'A08O' 'A0505']
['A27R' 'A27B' 'A27L' 'A27F' 'A27O' 'A2720210910"']
['주민' '사업자' '법인' '외국인' '기타' 'A2720210910"']
['A91DS' 'A90GS' 'A92LP' nan '7.0' 'A92CN' 'A94OT' 'A90GH' 'A93HD' 'A93EV'
 'A93SH' 'A92LH' 'A92LN' 'A91DH' 'A92CH' 'A91KS']
['경유' '휘발유' 'LPG(액화석유가스)' nan '7.0' 'CNG(압축천연가스)' '기타연료' '휘발유 하이브리드' '수소'
 '전기' '태양열' 'LPG 하이브리드' 'LNG(액화천연가스)' '경유 하이브리드' 'CNG 하이브리드' '등유']
['A30M' 'A30L' 'A30C' 'A30S' '0.0']
['중형' '대형' '경형' '소형' '0.0']


# 전처리

## 등록&제원

In [72]:
cs_before = carr.merge(srcr, on='제원관리번호', how='left')
cs_before.shape

(31131025, 26)

## 등록&제원&법정동

In [73]:
cs_before['법정동코드'].isnull().sum()

0

In [74]:
cs_before['법정동코드_수정'] = cs_before['법정동코드'].astype('str')
cs_before['법정동코드_수정'].head()

0    2714012300
1    2823710800
2    4111312700
3    3014011000
4    4713012500
Name: 법정동코드_수정, dtype: object

In [75]:
cs_before['법정동코드_수정'] = cs_before['법정동코드_수정'].str[:5] + '00000'
cs_before['법정동코드_수정'] = pd.to_numeric(cs_before['법정동코드_수정'], errors='coerce')
cs_before[['법정동코드', '법정동코드_수정']].head()

,법정동코드,법정동코드_수정
0,2714012300,2714000000
1,2823710800,2823700000
2,4111312700,4111300000
3,3014011000,3014000000
4,4713012500,4713000000


In [76]:
csc_before = cs_before.merge(coder, on='법정동코드', how='left')
csc_before.shape

(31131025, 31)

In [77]:
csc_before.columns

Index(['차대번호', '법정동코드', '배출가스등급', '배출가스인증번호', '차량말소여부', '제원관리번호', '차량연식', '차종',
       '용도', '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료',
       '차종유형', '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량',
       '엔진출력', '법정동코드_수정', '시도', '시군구', '시도코드', '시군구코드'],
      dtype='object')

In [78]:
csc_before[['시도', '시군구']].isnull().sum()

시도     23
시군구    49
dtype: int64

In [79]:
csc_before.loc[csc_before['시도'].isnull(), '법정동코드']

253143      5180031025
6001670     5180031033
7125734     5180031025
7852034     4165052000
8692073     5180031028
12526574    5180031028
12637600    5172035023
13739587    5175037022
13840652    5172035030
14915888    5180031023
15418353    5180031027
16291706    5180031031
16758757    5180031031
18306064    5180031023
18315819    5180031023
18402067    5180031023
18680038    5180031031
23466167    5172035031
30252963    4163055000
30721007    5172035024
30936586    5172035024
31086014    5180031023
31127292    5180031031
Name: 법정동코드, dtype: int64

In [80]:
csc_before_codey = csc_before[csc_before['시도'].notnull()]
csc_before_coden = csc_before[csc_before['시도'].isnull()]
csc_before_codey.shape[0] + csc_before_coden.shape[0] == csc_before.shape[0]

True

In [81]:
csc_before_coden.shape

(23, 31)

In [82]:
csc_before_coden = csc_before_coden.drop(['법정동코드', '시도', '시군구', '시도코드', '시군구코드'], axis=1)
csc_before_coden.shape

(23, 26)

In [83]:
csc_before_codenm = csc_before_coden.merge(coder, left_on='법정동코드_수정', right_on='법정동코드', how='left')
csc_before_codenm.shape

(23, 31)

In [84]:
csc_before_codenm.columns

Index(['차대번호', '배출가스등급', '배출가스인증번호', '차량말소여부', '제원관리번호', '차량연식', '차종', '용도',
       '최초등록일자', '제작일자', '차량관리번호', '자동차등록번호', '검사유효일', '소유자구분', '연료', '차종유형',
       '제작사명', '차명', '자동차형식', '엔진형식', '차종분류', '총중량', '적재중량', '배기량', '엔진출력',
       '법정동코드_수정', '법정동코드', '시도', '시군구', '시도코드', '시군구코드'],
      dtype='object')

In [85]:
csc_before_codenm[['시도', '시군구']].isnull().sum()

시도     0
시군구    0
dtype: int64

In [86]:
csc_before = pd.concat([csc_before_codey, csc_before_codenm], ignore_index=True)
csc_before.shape # 31131025

(31131025, 31)

In [87]:
csc_before[['시도', '시군구']].isnull().sum()

시도      0
시군구    26
dtype: int64

In [88]:
csc_before.loc[csc_before['시군구'].isnull(), ['시도', '시군구']]

,시도,시군구
747535,세종특별자치시,NaN
770310,세종특별자치시,NaN
1641011,경기도,NaN
1800621,세종특별자치시,NaN
5992843,세종특별자치시,NaN
6854442,세종특별자치시,NaN
6886850,세종특별자치시,NaN
8005829,세종특별자치시,NaN
8387500,세종특별자치시,NaN
8961169,세종특별자치시,NaN


### [출력] 등록&제원

In [89]:
today_date = datetime.today().strftime('%Y%m%d')
today_date

'20240104'

In [90]:
# 2m 55.7s
csc_before.to_csv(os.path.join(pub_fold, f'등록_제원_법정동_전처리완료_{today_date}.csv'), index=False)

In [91]:
# 차량말소 제거 데이터프레임 생성
cscm_before = csc_before[csc_before['차량말소여부'] == 'N'].reset_index(drop=True)
cscm_before.shape

(25895547, 31)

In [92]:
csc_before.shape, cscm_before.shape

((31131025, 31), (25895547, 31))

## LST

### [출력] LST

In [69]:
# 2m 55.3s
cs.to_csv(os.path.join(pub_fold, f'CEG_GRD_LST_전처리완료_{today_date}.csv'), index=False)

In [16]:
# 차량말소 제거 데이터프레임 생성
csm= cs[cs['차량말소여부'] == 'N'].reset_index(drop=True)
csm.shape

(25052895, 38)

In [17]:
cs.shape, csm.shape

((29709609, 38), (25052895, 38))

# 분석

## 차량 현황
- 말소여부, 등급, 지역, 연료별 현황

### 등록&제원

In [95]:
grp1_before = csc_before.groupby(['차량말소여부', '시도'], dropna=False)['차대번호'].count().reset_index()
grp1_before.loc[grp1_before.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp1_before['차대번호'].sum()]
grp1_before = grp1_before.rename(columns={'차대번호':'차대번호개수'})
grp1_before # 31131025

,차량말소여부,시도,차대번호개수
0,N,강원특별자치도,865371.00
1,N,경기도,6505925.00
2,N,경상남도,1938726.00
3,N,경상북도,1531987.00
4,N,광주광역시,721902.00
5,N,대구광역시,1256186.00
6,N,대전광역시,720579.00
7,N,부산광역시,1524100.00
8,N,서울특별시,3190988.00
9,N,세종특별자치시,197786.00


In [97]:
grp2_before = csc_before.groupby(['차량말소여부', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp2_before.loc[grp2_before.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp2_before['차대번호'].sum()]
grp2_before = grp2_before.rename(columns={'차대번호':'차대번호개수'})
grp2_before

,차량말소여부,배출가스등급,차대번호개수
0,N,1,3260843.00
1,N,2,10929631.00
2,N,3,8185490.00
3,N,4,1335156.00
4,N,5,923641.00
5,N,NaN,1260786.00
6,Y,1,171488.00
7,Y,2,1323958.00
8,Y,3,1739554.00
9,Y,4,571754.00


In [103]:
grp3_before = csc_before.groupby(['차량말소여부', '시도', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
grp3_before

배출가스등급,차량말소여부,시도,1,2,3,4,5,NaN
0,N,강원특별자치도,93671,353126,295897,48886,36684,37107
1,N,경기도,806161,2787982,2115473,313682,187428,295199
2,N,경상남도,216827,840372,600795,105397,82534,92801
3,N,경상북도,146821,623900,505781,105788,95392,54305
4,N,광주광역시,97038,314995,227003,34164,20757,27945
5,N,대구광역시,156603,543240,395836,63778,41167,55562
6,N,대전광역시,96592,321374,216592,36185,20225,29611
7,N,부산광역시,201128,644084,455874,68342,51589,103083
8,N,서울특별시,461096,1379431,945640,156575,106791,141455
9,N,세종특별자치시,29756,90447,58957,7113,3319,8194


In [105]:
grp4_before = csc_before.groupby(['차량말소여부', '연료', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
grp4_before

배출가스등급,차량말소여부,연료,1,2,3,4,5,NaN
0,N,CNG 하이브리드,269.00,17.00,NaN,NaN,NaN,NaN
1,N,CNG(압축천연가스),NaN,22139.00,3905.00,1.00,7.00,2092.00
2,N,LNG(액화천연가스),NaN,2.00,NaN,NaN,NaN,24.00
3,N,LPG 하이브리드,12559.00,NaN,NaN,NaN,NaN,2.00
4,N,LPG(액화석유가스),447980.00,1117641.00,181145.00,23747.00,429.00,33318.00
5,N,경유,NaN,NaN,7298500.00,1004248.00,893733.00,343049.00
6,N,경유 하이브리드,NaN,NaN,18499.00,NaN,NaN,29660.00
7,N,기타연료,NaN,NaN,5.00,34.00,20.00,73846.00
8,N,등유,NaN,NaN,NaN,NaN,NaN,3.00
9,N,수소,33951.00,NaN,NaN,NaN,NaN,28.00


### LST

In [ ]:
grp1 = cs.groupby(['차량말소여부', '시도'], dropna=False)['차대번호'].count().reset_index()
grp1.loc[grp1.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp1['차대번호'].sum()]
grp1 = grp1.rename(columns={'차대번호':'차대번호개수'})
grp1

In [ ]:
grp2 = cs.groupby(['차량말소여부', '배출가스등급'], dropna=False)['차대번호'].count().reset_index()
grp2.loc[grp2.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp2['차대번호'].sum()]
grp2 = grp2.rename(columns={'차대번호':'차대번호개수'})
grp2

In [ ]:
grp3 = cs.groupby(['차량말소여부', '시도', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
grp3.loc[grp3.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp3['차대번호'].sum()]
grp3 = grp3.rename(columns={'차대번호':'차대번호개수'})
grp3

In [ ]:
grp4 = cs.groupby(['차량말소여부', '연료', '배출가스등급'], dropna=False)['차대번호'].count().unstack('배출가스등급').reset_index()
grp4.loc[grp4.shape[0], ['차량말소여부', '차대번호']] = ['합계', grp4['차대번호'].sum()]
grp4 = grp4.rename(columns={'차대번호':'차대번호개수'})
grp4